<a href="https://www.kaggle.com/code/shubhamgupta012/weed-detection-using-autoencoders?scriptVersionId=129641272" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/1259.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/659.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/274.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/315.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/919.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/2037.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/1819.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/1938.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/948.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/123.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/2939.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/959.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/2824.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/soil/1724.tif
/kaggle/input/weed-detection-in-soybean-crops/dataset/s

In [2]:
import tensorflow as tf
import cv2
import os
import numpy as n
import glob

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Define directory paths
base_dir = '/kaggle/input/weed-detection-in-soybean-crops/dataset'
categories = ['broadleaf', 'grass', 'soil', 'soybean']
base_dir1 = '/kaggle/working/TheCreatedFolder'

In [4]:
# for cl in categories:
#   img_path = os.path.join(base_dir, cl)
#   images = glob.glob(img_path + '/*.tif')
#   print("{}: {} Images".format(cl, len(images)))
#   num_train = int(round(len(images)*0.8))
#   train, val = images[:num_train], images[num_train:]

#   for t in train:
#     if not os.path.exists(os.path.join(base_dir, 'train', cl)):
#       os.makedirs(os.path.join(base_dir, 'train', cl))
#     shutil.move(t, os.path.join(base_dir, 'train', cl))

#   for v in val:
#     if not os.path.exists(os.path.join(base_dir, 'val', cl)):
#       os.makedirs(os.path.join(base_dir, 'val', cl))
#     shutil.move(v, os.path.join(base_dir, 'val', cl))

In [5]:
# Define image size
img_size = (200, 200)

# Create empty lists for image and label data
images = []
labels = []

In [6]:
# Iterate over categories and image files
for i, category in enumerate(categories):
    category_dir = os.path.join(base_dir, category)
    for filename in os.listdir(category_dir):
        if filename.endswith('.tif'):
            # Read image file and resize to target size
            image = cv2.imread(os.path.join(category_dir, filename))
            image = cv2.resize(image, img_size)
            # Append image and label data to lists
            images.append(image)
            labels.append(i)

In [7]:
# Convert image and label data to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Define the input shape
input_shape = (img_size[0], img_size[1], 3)

In [8]:
# Define the encoder
x = tf.keras.layers.Input(shape=input_shape)
ec1 = tf.keras.layers.Conv2D(20, 5, activation='relu', padding='same')(x)
ec2 = tf.keras.layers.Conv2D(20, 5, activation='relu', padding='same')(ec1)
pool1 = tf.keras.layers.MaxPooling2D()(ec2)
ec3 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(pool1)
ec4 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(ec3)
pool2 = tf.keras.layers.MaxPooling2D()(ec4)
ec5 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(pool2)
ec6 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(ec5)
pool3 = tf.keras.layers.MaxPooling2D()(ec6)

# Define the decoder
dc1 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(pool3)
dc2 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(dc1)
samp1 = tf.keras.layers.UpSampling2D()(dc2)
dc3 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(samp1)
dc4 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(dc3)
samp2 = tf.keras.layers.UpSampling2D()(dc4)
dc5 = tf.keras.layers.Conv2D(10, 5, activation='relu', padding='same')(samp2)
dc6 = tf.keras.layers.Conv2D(4, 5, activation='softmax', padding='same')(dc5)
samp3 = tf.keras.layers.UpSampling2D()(dc6)

In [9]:
classification_output = tf.keras.layers.Conv2D(4, 5, activation='softmax', padding='same')(samp3)

# Create the classification model
classification_model = tf.keras.models.Model(inputs=x, outputs=classification_output)

# Compile the model
classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
classification_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 200, 200, 20)      1520      
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 20)      10020     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 20)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 10)      5010      
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 10)      2510      
                                                             

In [10]:
# epochs = 80

# history = classification_model.fit(
#     images,
#     labels,
# #     steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
#     epochs=epochs,
# #     validation_data=val_data_gen,
# #     validation_steps=int(np.ceil(val_data_gen.n / float(batch_size)))
# )